In [1]:
import praw
import pandas as pd
import logging
from datetime import datetime, timezone, timedelta
import os
import time
import re

In [2]:
reddit = praw.Reddit(user_agent = True, 
client_id = "DSPYqJYKgnkPWC1hYN8Cnw", 
client_secret = "2HlMZf-X1PnwsoyfsMvNvwvBGn6FrA", 
username = "Lost-Recognition2843",
password = "Ihopeyouhaveagreatday1219")

In [3]:
def fetch_daily_posts_with_content_df(subreddits, keywords, start_date, end_date, limit=20):
    """
    Fetch exactly 'limit' posts per day per subreddit, restricted to those containing
    at least one keyword, and only collect comments containing the keywords as well.
    
    Parameters:
        subreddits : list of subreddit names (e.g. ["technology", "science"])
        keywords   : list of keywords to search (e.g. ["gemini", "google"])
        start_date : datetime object of the start date
        end_date   : datetime object of the end date
        limit      : number of posts to fetch per day, per subreddit

    Returns:
        pd.DataFrame with columns: ["date", "subreddit", "post", "main"]
    """
    # Build a single query string that uses OR to include any of the keywords
    # e.g. keywords = ["gemini", "google"] => "gemini OR google"
    search_query = " OR ".join(keywords)
    
    # Regex pattern (case-insensitive) to ensure text actually contains at least one keyword
    # e.g. if keywords = ["gemini", "google"], pattern => r"(?i)gemini|google"
    pattern = re.compile("|".join(keywords), re.IGNORECASE)

    data = []
    current_date = start_date

    while current_date <= end_date:
        date_start = int(current_date.timestamp())
        date_end = int((current_date + timedelta(days=1)).timestamp())

        for subreddit_name in subreddits:
            count = 0  # Track posts collected for the subreddit on this day
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Searching r/{subreddit_name} on {current_date.strftime('%Y-%m-%d')}...")

            try:
                # Search for posts that contain any of our keywords
                for submission in subreddit.search(search_query, sort="relevant", time_filter="all"):
                    # Check if it falls within today's date range
                    if date_start <= submission.created_utc < date_end:
                        # Check if we already have enough posts for the day
                        if count >= limit:
                            break

                        # Double-check the submission actually contains one of our keywords
                        text_to_check = (submission.title or "") + "\n" + (submission.selftext or "")
                        if not pattern.search(text_to_check):
                            # If the submission itself doesn't match, skip
                            continue

                        # Capture the post content (or note if it's a link post)
                        post_content = submission.selftext if submission.selftext else "No content (link post)"

                        # Fetch top-level comments (limit to 10), strictly within the same day
                        submission.comments.replace_more(limit=0)
                        top_comments = [
                            comment
                            for comment in submission.comments.list()[:10]
                            if date_start <= comment.created_utc < date_end
                        ]

                        print(top_comments)

                        # Now only collect comments that also have at least one keyword
                        for comment in top_comments:
                            print(comment.body, pattern.search(comment.body))
                            if pattern.search(comment.body):
                                comment_dt = datetime.utcfromtimestamp(comment.created_utc)
                                print("✅", comment.body)
                                data.append({
                                    "date": comment_dt.strftime('%Y-%m-%d'),
                                    "subreddit": subreddit_name,
                                    "post": comment.body,
                                    "main": post_content
                                })

                        count += 1  # Count this post toward our daily limit

            except Exception as e:
                print(f"Error fetching from r/{subreddit_name}: {e}")

        current_date += timedelta(days=1)

    # Convert the list of posts to a DataFrame
    df = pd.DataFrame(data, columns=["date", "subreddit", "post", "main"])
    return df

## Google

In [4]:
subreddits = ['google', "GoogleOne", "technology", "stocks", "investing", "wallstreetbets", "ValueInvesting", "finance", "technology", "StocksAndTrading"]
keywords = ["gemini", "google", "pixel", "gates", "bill", "GOOG", 'Alphabet', "Search", "AI"]
# keyword = 'A'
start_date = datetime(2023, 6, 1, tzinfo=timezone.utc)
end_date = datetime(2025, 2, 3, tzinfo=timezone.utc)
daily_limit = 20  # Get exactly 5 posts per subreddit per day
results_df = fetch_daily_posts_with_content_df(subreddits, keywords, start_date, end_date, daily_limit)

Searching r/google on 2023-06-01...
Searching r/GoogleOne on 2023-06-01...
Searching r/technology on 2023-06-01...
Searching r/stocks on 2023-06-01...
Searching r/investing on 2023-06-01...
Searching r/wallstreetbets on 2023-06-01...
Searching r/ValueInvesting on 2023-06-01...
Searching r/finance on 2023-06-01...
Searching r/technology on 2023-06-01...
Searching r/StocksAndTrading on 2023-06-01...
Searching r/google on 2023-06-02...
Searching r/GoogleOne on 2023-06-02...
Searching r/technology on 2023-06-02...
Searching r/stocks on 2023-06-02...
Searching r/investing on 2023-06-02...
Searching r/wallstreetbets on 2023-06-02...
Searching r/ValueInvesting on 2023-06-02...
Searching r/finance on 2023-06-02...
Searching r/technology on 2023-06-02...
Searching r/StocksAndTrading on 2023-06-02...
Searching r/google on 2023-06-03...
Searching r/GoogleOne on 2023-06-03...
Searching r/technology on 2023-06-03...
Searching r/stocks on 2023-06-03...
Searching r/investing on 2023-06-03...
Searchin

/var/folders/df/z_1lzxjn20s6k22x8_q5h3br0000gn/T/ipykernel_4327/2314012705.py:68: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_dt = datetime.utcfromtimestamp(comment.created_utc)


Searching r/finance on 2023-06-08...
Searching r/technology on 2023-06-08...
Searching r/StocksAndTrading on 2023-06-08...
[Comment(id='jneq9k9')]
🚀 🌑  -- Join our discord!! https://discord.gg/jcewXNmf6C -- 🚀 🌑

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/StocksAndTrading) if you have any questions or concerns.* None
Searching r/google on 2023-06-09...
Searching r/GoogleOne on 2023-06-09...
Searching r/technology on 2023-06-09...
Searching r/stocks on 2023-06-09...
Searching r/investing on 2023-06-09...
Searching r/wallstreetbets on 2023-06-09...
Searching r/ValueInvesting on 2023-06-09...
Searching r/finance on 2023-06-09...
Searching r/technology on 2023-06-09...
Searching r/StocksAndTrading on 2023-06-09...
Searching r/google on 2023-06-10...
Searching r/GoogleOne on 2023-06-10...
Searching r/technology on 2023-06-10...
Searching r/stocks on 2023-06-10...
Searching r/investing on 2023-06-10...
Se

In [5]:
results_df

,date,subreddit,post,main
0,2023-06-08,wallstreetbets,I just applied for the AI position. Fingers cr...,No content (link post)
1,2023-06-08,wallstreetbets,"""speak clearly"" \n\nThis ain't gonna work at T...",No content (link post)
2,2023-06-08,wallstreetbets,Jim Kramer said AI would replace $24 an hour h...,No content (link post)
3,2023-06-08,wallstreetbets,I inquired about the job opportunities mention...,No content (link post)
4,2023-06-08,wallstreetbets,Gonna be a long time before people understand ...,No content (link post)
...,...,...,...,...
1697,2025-02-03,technology,"It definitely punctured the hype train, but th...",No content (link post)
1698,2025-02-03,technology,And here I thought it was because they've been...,No content (link post)
1699,2025-02-03,technology,This is just an interesting story overall. And...,No content (link post)
1700,2025-02-03,technology,Hundreds of billions have been thrown at vario...,No content (link post)


In [9]:
# os.makedirs("../CAPSTONE-stockreturn/data/sentiment", exist_ok=True)

# Now export the DataFrame to CSV
df_to_export.to_csv("reddit_sentiment_data.csv", index=False)

## CSV

In [10]:
subreddits = ['cvs', "CVS_Workers", "pharmacy", "stocks", "investing", "wallstreetbets", "ValueInvesting", "finance", "flu", "StocksAndTrading"]
keywords = ["cvs", "pharmacy", "healthcare", "prescription", "wellness"]
# keyword = 'A'
start_date = datetime(2023, 6, 1, tzinfo=timezone.utc)
end_date = datetime(2025, 2, 3, tzinfo=timezone.utc)
daily_limit = 20  # Get exactly 5 posts per subreddit per day
results_df = fetch_daily_posts_with_content_df(subreddits, keywords, start_date, end_date, daily_limit)

Searching r/cvs on 2023-06-01...
Searching r/CVS_Workers on 2023-06-01...
Searching r/pharmacy on 2023-06-01...
Searching r/stocks on 2023-06-01...
Searching r/investing on 2023-06-01...
Searching r/wallstreetbets on 2023-06-01...
Searching r/ValueInvesting on 2023-06-01...
Searching r/finance on 2023-06-01...
Searching r/flu on 2023-06-01...
Searching r/StocksAndTrading on 2023-06-01...
Searching r/cvs on 2023-06-02...
Searching r/CVS_Workers on 2023-06-02...
Searching r/pharmacy on 2023-06-02...
Searching r/stocks on 2023-06-02...
Searching r/investing on 2023-06-02...
Searching r/wallstreetbets on 2023-06-02...
Searching r/ValueInvesting on 2023-06-02...
Searching r/finance on 2023-06-02...
Searching r/flu on 2023-06-02...
Searching r/StocksAndTrading on 2023-06-02...
Searching r/cvs on 2023-06-03...
Searching r/CVS_Workers on 2023-06-03...
Searching r/pharmacy on 2023-06-03...
Searching r/stocks on 2023-06-03...
Searching r/investing on 2023-06-03...
Searching r/wallstreetbets on 2

/var/folders/df/z_1lzxjn20s6k22x8_q5h3br0000gn/T/ipykernel_4327/2314012705.py:68: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_dt = datetime.utcfromtimestamp(comment.created_utc)


Searching r/investing on 2023-06-19...
Searching r/wallstreetbets on 2023-06-19...
Searching r/ValueInvesting on 2023-06-19...
Searching r/finance on 2023-06-19...
Searching r/flu on 2023-06-19...
Searching r/StocksAndTrading on 2023-06-19...
Searching r/cvs on 2023-06-20...
Searching r/CVS_Workers on 2023-06-20...
Searching r/pharmacy on 2023-06-20...
Searching r/stocks on 2023-06-20...
Searching r/investing on 2023-06-20...
Searching r/wallstreetbets on 2023-06-20...
Searching r/ValueInvesting on 2023-06-20...
Searching r/finance on 2023-06-20...
Searching r/flu on 2023-06-20...
Searching r/StocksAndTrading on 2023-06-20...
Searching r/cvs on 2023-06-21...
Searching r/CVS_Workers on 2023-06-21...
Searching r/pharmacy on 2023-06-21...
Searching r/stocks on 2023-06-21...
Searching r/investing on 2023-06-21...
Searching r/wallstreetbets on 2023-06-21...
Searching r/ValueInvesting on 2023-06-21...
Searching r/finance on 2023-06-21...
Searching r/flu on 2023-06-21...
Searching r/StocksAnd

In [11]:
results_df

,date,subreddit,post,main
0,2023-06-19,pharmacy,I mean.... people just need to stop coming in....,"As you may already know, CVS cut its hours of ..."
1,2023-06-19,pharmacy,Charities have volunteers and the last I check...,"As you may already know, CVS cut its hours of ..."
2,2023-06-19,pharmacy,All these pharmacists working for free are hur...,"As you may already know, CVS cut its hours of ..."
3,2023-07-09,stocks,Iv been and will keep adding to CVS and CI. I ...,"Lately, I am paying very close attention to he..."
4,2023-07-09,stocks,CVS also owns Aetna,"Lately, I am paying very close attention to he..."
...,...,...,...,...
385,2025-01-24,cvs,"CVS badly needs a strong, non-captured union.",Not only did we lose the best pharmacist becau...
386,2025-01-24,cvs,I left CVS went to an independent. They should...,Not only did we lose the best pharmacist becau...
387,2025-01-26,pharmacy,Now they have one extra thing to bitch about a...,No content (link post)
388,2025-01-26,pharmacy,CVS nearby,No content (link post)


In [12]:
df_to_export = results_df[['date', 'subreddit', 'post']]
df_to_export.to_csv("reddit_sentiment_data_cvs.csv", index=False)